## Model

In [ ]:
objective_used = skll_objective if skll_objective else 'neg_mean_squared_error'
intro_markdown_str = f"""### Characteristics 

- Model: **{model_name}**
- Number of features: **{len(features_used)}**
- Tuning objective: **{objective_used}**
"""

if skll_fixed_parameters:
    intro_markdown_str += "- Custom fixed parameters:"
    for parameter, value in skll_fixed_parameters.items():
        intro_markdown_str += f"\n    - `{parameter}` : **{value!r}**"

display(Markdown(intro_markdown_str))

In [ ]:
skll_linear_models = ['Ridge', 'BayesianRidge', 'LinearSVR', 'Lasso', 'Lars', 'HuberRegressor', 'TheilSenRegressor']

In [ ]:
weights_markdown_str = """### Feature weights

Here are the feature weights as learned by the model."""

if model_name in skll_linear_models:
    display(Markdown(weights_markdown_str))

In [ ]:
if model_name in skll_linear_models:

    from rsmtool.modeler import Modeler
    from six import iteritems
    from sklearn.svm import SVR
    
    model_file = join(output_dir, '{}.model'.format(experiment_id))
    learner = Modeler.load_from_file(model_file).learner
    
    # get the coefficients and the intercept
    weights = {}
    coef = learner.model.coef_
    intercept = {'_intercept_': learner.model.intercept_}

    # convert SVR coefficient format (1 x matrix) to array
    if isinstance(learner._model, SVR):
        coef = coef.toarray()[0]

    # inverse transform to get indices for before feature selection
    coef = learner.feat_selector.inverse_transform(coef.reshape(1, -1))[0]
    for feat, idx in iteritems(learner.feat_vectorizer.vocabulary_):
        if coef[idx]:
            weights[feat] = coef[idx]

    # Some learners (e.g. LinearSVR) may return a list of intercepts
    if isinstance(intercept['_intercept_'], np.ndarray):
        intercept_list = ["%.12f" % i for i in intercept['_intercept_']]
        print("intercept = {}".format(intercept_list))
    else:
        print("intercept = {:.12f}".format(intercept['_intercept_']))
    print()
        
    print("Number of nonzero features:", len(weights))
    weight_items = iteritems(weights)
    for feat, val in sorted(weight_items, key=lambda x: -abs(x[1])):
        print("{:.12f}\t{}".format(val, feat))